# Post-Quantum Encrypted File Sharing System

## Introduction
With the advent of quantum computing, traditional encryption algorithms such as RSA and ECC are at risk of being broken using quantum algorithms like Shor’s algorithm. To future-proof sensitive data, Post-Quantum Cryptography (PQC) offers cryptographic schemes that remain secure against quantum attacks. One such approach is Lattice-Based Cryptography, particularly Ring Learning With Errors (Ring-LWE), which provides robust security and efficiency.

## Objective
The objective of this project is to develop a secure file encryption tool leveraging Ring-LWE encryption, ensuring resistance against both classical and quantum attacks. The system will:

- Generate Post-Quantum Keys using the Ring-LWE problem.
- Encrypt Files Securely by computing a shared secret using the generated keys.
- Decrypt Files Correctly by reconstructing the shared secret and recovering the original data.
- Enable Secure Key Management for safe storage and retrieval of encryption keys.

This project serves as a real-world implementation of post-quantum cryptographic techniques, contributing to research in quantum-resistant encryption. It can be extended to develop secure cloud storage solutions, encrypted messaging applications, or quantum-safe communication protocols.

## Future Scope
- Integration with Digital Signatures to verify data integrity and authenticity.
- Implementation of Key Exchange Protocols for secure multi-user communication.
- Optimizations for Performance to handle large files efficiently.

### Post-Quantum Cryptography (PQC)
Post-Quantum Cryptography refers to cryptographic algorithms that are secure against attacks by quantum computers. Classical cryptographic schemes like RSA, ECC, and Diffie-Hellman rely on mathematical problems (such as integer factorization and discrete logarithms) that can be efficiently solved using quantum algorithms like Shor’s Algorithm. To ensure long-term security, researchers are developing new encryption methods based on problems that even quantum computers cannot easily solve.

### Lattice-Based Cryptography
Lattice-based cryptography is one of the most promising areas in PQC. It is based on the hardness of problems related to lattices, which are grid-like mathematical structures in multidimensional space. The difficulty of solving problems like the Shortest Vector Problem (SVP) and the Learning With Errors (LWE) problem makes lattice-based cryptography highly resistant to attacks, even from quantum computers.

#### Learning With Errors (LWE) and Ring-LWE
- **LWE:** The Learning With Errors problem is a mathematical problem that involves solving a system of linear equations where small errors (noise) are added. Finding the solution is computationally hard, making it an excellent foundation for encryption schemes.
- **Ring-LWE:** A more efficient variant of LWE that works within a structured algebraic framework (rings), improving performance while maintaining security. It is used in real-world post-quantum encryption schemes like Kyber and NTRUEncrypt.

In [ ]:
!pip install pycryptodome kyber_py

## Working

1. Generate Kyber-512 keys.

2. Perform Key Encapsulation Mechanism (KEM) using Kyber-512.

3. Encrypt the file using AES-256.

4. Decrypt the file using AES-256.

5. Compare the original and decrypted file to verify correctness.

6. Display all intermediate steps and outputs for clarity.

In [ ]:
import os
import hashlib
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from kyber_py.kyber import Kyber512

def generate_kyber_keys():
    print("\nGenerating Kyber-512 Key Pair")
    public_key, secret_key = Kyber512.keygen()
    print("Public Key Generated: ", public_key.hex()[:64], "...")
    print("Secret Key Generated: ", secret_key.hex()[:64], "...\n")
    return public_key, secret_key

def key_encapsulation(public_key):
    print("Performing Key Encapsulation")
    shared_secret, ciphertext = Kyber512.encaps(public_key)
    print("Shared Secret Encapsulated: ", shared_secret.hex()[:64], "...")
    print("Ciphertext (Encrypted Key): ", ciphertext.hex()[:64], "...\n")
    return shared_secret, ciphertext

def key_decapsulation(secret_key, ciphertext):
    print("Decapsulating the Shared Secret")
    recovered_secret = Kyber512.decaps(secret_key, ciphertext)
    print("Recovered Shared Secret: ", recovered_secret.hex()[:64], "...\n")
    return recovered_secret

def encrypt_file(file_path, key):
    print("Encrypting the File using AES-256")
    cipher = AES.new(key, AES.MODE_CBC)

    with open(file_path, 'rb') as file:
        data = file.read()

    encrypted_data = cipher.encrypt(pad(data, AES.block_size))
    enc_file_path = file_path + ".enc"

    with open(enc_file_path, 'wb') as enc_file:
        enc_file.write(cipher.iv)  # Store Initialization Vector (IV) at the beginning
        enc_file.write(encrypted_data)

    print("File Encrypted Successfully. Encrypted File Path: ", enc_file_path, "\n")
    return enc_file_path

def decrypt_file(file_path, key):
    print("Decrypting the Encrypted File")

    with open(file_path, 'rb') as enc_file:
        iv = enc_file.read(16)  # Read IV from the file
        encrypted_data = enc_file.read()

    cipher = AES.new(key, AES.MODE_CBC, iv=iv)
    decrypted_data = unpad(cipher.decrypt(encrypted_data), AES.block_size)

    decrypted_file_path = file_path.replace(".enc", "_decrypted")
    with open(decrypted_file_path, 'wb') as dec_file:
        dec_file.write(decrypted_data)

    print("File Decryption Completed. Decrypted File Path: ", decrypted_file_path, "\n")
    return decrypted_file_path

def compare_files(file1, file2):
    print("Verifying File Integrity")

    with open(file1, 'rb') as f1, open(file2, 'rb') as f2:
        original_data = f1.read()
        decrypted_data = f2.read()

    if original_data == decrypted_data:
        print("Verification Successful: The decrypted file matches the original.\n")
        return True
    else:
        print("Verification Failed: The decrypted file does not match the original.\n")
        return False

def main():
    # Get file path from user
    file_path = input("Enter the file path to encrypt & decrypt: ")

    if not os.path.exists(file_path):
        print("Error: The specified file does not exist.")
        return

    # Step 1: Generate Kyber-512 Key Pair
    public_key, secret_key = generate_kyber_keys()

    # Step 2: Perform Key Encapsulation (Encrypts shared secret)
    shared_secret, ciphertext = key_encapsulation(public_key)

    # Step 3: Perform Key Decapsulation (Decrypts shared secret)
    recovered_secret = key_decapsulation(secret_key, ciphertext)

    # Verify if the shared secrets match
    assert shared_secret == recovered_secret, "Key Exchange Failed. Shared secrets do not match."

    # Extract AES Key from the shared secret
    aes_key = shared_secret[:32]  # First 32 bytes used for AES-256
    aes_key_hash = hashlib.sha256(aes_key).hexdigest()  # Hash for display
    print("AES-256 Encryption Key (SHA-256 Hash): ", aes_key_hash, "\n")

    # Step 4: Encrypt the File
    encrypted_file = encrypt_file(file_path, aes_key)

    # Step 5: Decrypt the File
    decrypted_file = decrypt_file(encrypted_file, aes_key)

    # Step 6: Compare and Verify File Integrity
    compare_files(file_path, decrypted_file)

if __name__ == "__main__":
    main()

Enter the file path to encrypt & decrypt: /content/secret.txt

Generating Kyber-512 Key Pair
Public Key Generated:  b2d6085fa43817074408a8050c45cd904b4f4aac91b9d1b20b317c6ae8052773 ...
Secret Key Generated:  7d9baf42e1c25b10aa36fa9cc85b4126808b050322b1f616cec519c24954a351 ...

Performing Key Encapsulation
Shared Secret Encapsulated:  e225401a472deec8793f962a71fbf21908c9177a89af059da6a2bda6056d1959 ...
Ciphertext (Encrypted Key):  ba567ba90f56d488e8b450fb20afee21bd71beac31b587f19faee7516eb4c3e4 ...

Decapsulating the Shared Secret
Recovered Shared Secret:  e225401a472deec8793f962a71fbf21908c9177a89af059da6a2bda6056d1959 ...

AES-256 Encryption Key (SHA-256 Hash):  cc2c33f3415e5eb8b47f630a4797e90f4b3f022fd4ee7dbfdfd1f429f8406a22 

Encrypting the File using AES-256
File Encrypted Successfully. Encrypted File Path:  /content/secret.txt.enc 

Decrypting the Encrypted File
File Decryption Completed. Decrypted File Path:  /content/secret.txt_decrypted 

Verifying File Integrity
Verification S